In [7]:
%cd ~/kaggle_nlp/challenge_files/

/home/infres/amathur-23/kaggle_nlp/challenge_files


/home/infres/amathur-23/kaggle_nlp/challenge_files/nlp2/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/infres/amathur-23/kaggle_nlp/challenge_files/nlp2/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [13]:
# load model and tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM

import pandas as pd
val_df = pd.read_csv("data/validation.csv")


In [14]:
def get_headline(model_path, text):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    if 't5' in model_path : text = "summarize: " + text 
    inputs = tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=50, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [15]:
text = val_df['text'][0]
title = val_df['titles'][0]



title_camembert = get_headline("camembert_enc_dec", text)
title_t5 = get_headline("t5_small_train_out", text)

The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


In [5]:
NLI_CONFIG = "cmarkea/distilcamembert-base-nli"
# NLI_CONFIG = "mtheo/camembert-base-xnli"
nli_model = AutoModelForSequenceClassification.from_pretrained(NLI_CONFIG).to(device)
nli_tokenizer = AutoTokenizer.from_pretrained(NLI_CONFIG) 

tensor([0.8641], grad_fn=<SelectBackward0>)

In [21]:
import torch
def get_entailment(nli_model, nli_tokenizer, text, pred):
    nli_model.eval()
    with torch.no_grad():
        text = " ".join(text.split(" ")[:512])
        x = nli_tokenizer.encode(text, pred, return_tensors='pt', max_length=512)#.to(device)
        # print(x.shape)
        logits = nli_model(x)[0]
        entail_contradiction_logits = logits[:,::2]
        probs = entail_contradiction_logits.softmax(dim=1)
        entailment = probs[:,0].detach().cpu().numpy()[0]
    return entailment

In [31]:
from rouge_score import rouge_scorer

def get_rouge(text, title):
    scorer = rouge_scorer.RougeScorer(['rougeL'])
    rouge_l_f1 = scorer.score(text, title)['rougeL'][2]
    return rouge_l_f1

In [22]:
print(text)

Sur les réseaux sociaux, les images sont impressionnantes. Dimanche matin à Venise, l'équipage du MSC Opéra a perdu le contrôle du paquebot, à son arrivée dans le port de la cité des Doges. Le navire, qui peut contenir plus de 2.600 passagers, est venu heurter le quai auquel il voulait s'arrimer. Le paquebot a raclé le quai sur plusieurs mètres, suscitant la panique des personnes à terre, avant de percuter un autre bateau touristique, le Michelangelo, stoppant ainsi sa course. Des témoins ont filmé la scène. Les vidéos montrent des touristes courant pour tenter de fuir le paquebot, qui ne semble pas vouloir s'arrêter. Quatre personnes ont été blessées dans cet accident : deux légèrement, tandis que les deux autres ont été transportées à l'hôpital pour des examens. L'incident s'est produit à San Basilio-Zaterre, dans le canal de la Giudecca, où de nombreux navires de croisière s'arrêtent pour permettre à leurs passagers de visiter Venise.Selon le quotidien italien Corriere della Serra, 

In [23]:
print(title)
print(get_entailment(nli_model, nli_tokenizer, text, title))

Le bateau de croisière, long de 275 m, a percuté un quai lors de son arrivée dans le port de Venise, dimanche 2 juin. Quatre personnes ont été blessées.
0.89850354


In [24]:
print(title_camembert)
print(get_entailment(nli_model, nli_tokenizer, text, title_camembert))

L'accident s'est produit à San Basilio-Zaterre, dans le sud de l'Italie. Quatre personnes ont été blessées, tandis que deux autres ont été transportées à l'hôpital.
0.9839316


In [25]:
print(title_t5)
print(get_entailment(nli_model, nli_tokenizer, text, title_t5))

L'équipage du MSC Opéra a perdu le contrôle du paquebot, à son arrivée dans le port de Venise. Des témoins ont filmé la scène.
0.99796355


In [27]:
premises = [
    "Le score pour les bleus est élevé", 
]*3

hypotheses = [
    "L'équipe de France a fait un bon match", 
    "l'équipe de France va sûrement perdre le match aujourd'hui.", 
    "La Seine a plus de 14 000 ans."
]

ents = []

for p, h in zip(premises, hypotheses):
    ents.append(get_entailment(nli_model, nli_tokenizer, p, h))

df = pd.DataFrame({'premise':premises, 'hypothesis':hypotheses, 'entailment_score':ents})
df.head()

,premise,hypothesis,entailment_score
0,Le score pour les bleus est élevé,L'équipe de France a fait un bon match,0.872316
1,Le score pour les bleus est élevé,l'équipe de France va sûrement perdre le match...,0.015618
2,Le score pour les bleus est élevé,La Seine a plus de 14 000 ans.,0.247390


In [28]:
latex_table = df.to_latex(index=False)
print(latex_table)

\begin{tabular}{llr}
\toprule
premise & hypothesis & entailment_score \\
\midrule
Le score pour les bleus est élevé & L'équipe de France a fait un bon match & 0.872316 \\
Le score pour les bleus est élevé & l'équipe de France va sûrement perdre le match aujourd'hui. & 0.015618 \\
Le score pour les bleus est élevé & La Seine a plus de 14 000 ans. & 0.247390 \\
\bottomrule
\end{tabular}



In [29]:
text

"Sur les réseaux sociaux, les images sont impressionnantes. Dimanche matin à Venise, l'équipage du MSC Opéra a perdu le contrôle du paquebot, à son arrivée dans le port de la cité des Doges. Le navire, qui peut contenir plus de 2.600 passagers, est venu heurter le quai auquel il voulait s'arrimer. Le paquebot a raclé le quai sur plusieurs mètres, suscitant la panique des personnes à terre, avant de percuter un autre bateau touristique, le Michelangelo, stoppant ainsi sa course. Des témoins ont filmé la scène. Les vidéos montrent des touristes courant pour tenter de fuir le paquebot, qui ne semble pas vouloir s'arrêter. Quatre personnes ont été blessées dans cet accident : deux légèrement, tandis que les deux autres ont été transportées à l'hôpital pour des examens. L'incident s'est produit à San Basilio-Zaterre, dans le canal de la Giudecca, où de nombreux navires de croisière s'arrêtent pour permettre à leurs passagers de visiter Venise.Selon le quotidien italien Corriere della Serra,

In [32]:
titles = [title, title_camembert, title_t5]
for t in titles:
    print(get_rouge(title, t))

1.0
0.28571428571428575
0.3278688524590164


In [33]:
titles

['Le bateau de croisière, long de 275 m, a percuté un quai lors de son arrivée dans le port de Venise, dimanche 2 juin. Quatre personnes ont été blessées.',
 "L'accident s'est produit à San Basilio-Zaterre, dans le sud de l'Italie. Quatre personnes ont été blessées, tandis que deux autres ont été transportées à l'hôpital.",
 "L'équipage du MSC Opéra a perdu le contrôle du paquebot, à son arrivée dans le port de Venise. Des témoins ont filmé la scène."]